# Coding agents from scratch

In [44]:
import os
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Callable

In [45]:
env = load_dotenv()
api_key: str = os.environ.get("OPENAI_API_KEY")

In [46]:
class LLMPrompt:
    def __init__(self, role, prompt_type, prompt_message):
        self.message = {
            "role": role,
            "content": [{
                "type": prompt_type,
                prompt_type: prompt_message
            }]
        }
            
class LLMAgentSystemPrompt(LLMPrompt):
    def __init__(self, system_prompt):
        super().__init__(role="developer", prompt_type="text", prompt_message=system_prompt)

In [55]:
class LLMClient:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def call(self, model, messages, max_completion_tokens=1000):
        response = self.client.chat.completions.create(
            model=model,
            messages=messages,
            max_completion_tokens=max_completion_tokens
        )
        return response.choices[0].message.content

class LLMAgent:
    def __init__(self, system_prompt, model, api_key):
        self.client = LLMClient(api_key=api_key)
        self.model = model
        self.system_prompt = LLMAgentSystemPrompt(system_prompt).message


In [56]:
class ChainerAgent(LLMAgent):
    def __init__(self, system_prompt, steps, inp, model, api_key):
        super().__init__(system_prompt, model, api_key)        
        self.steps = steps
        self.inp = inp
        self.model = model

    def chain(self, verbose=False):
        chain = [self.system_prompt]
        current_input = self.inp
        for ix, step in enumerate(self.steps):            
            new_step = LLMPrompt("user", "text", f"{step}\nInput: {current_input}").message
            chain.append(new_step)
            response_content = self.client.call(
                model=self.model,
                messages=chain,
                max_completion_tokens=1000)

            # Add the assistant's response to the chain
            assistant_response = LLMPrompt("assistant", "text", response_content).message
            chain.append(assistant_response)
            if verbose:
                print(f"STEP: {ix}")
                print(response_content)
                print("="*36)
            current_input = response_content
        return response_content


In [57]:
class BloggerAgent(ChainerAgent):
    def __init__(self, topic, target_audience, word_count, model="gpt-4o-mini", api_key=None):
        # Define the system prompt for blog post generation
        system_prompt = """You are a professional blog content creator who specializes in creating
        high-quality, engaging blog posts on various topics. You follow SEO best practices
        and create content that is both informative and engaging."""
        
        # Define the steps for the blog post creation pipeline
        steps = [
            # Step 1: Generate an outline
            """Create a detailed outline for a blog post on the given topic.
            Include a title, introduction, 4-6 main sections with subpoints, and conclusion.
            Format the outline with clear headings and bullet points.""",
            
            # Step 2: Validate and refine the outline
            """Review the outline and ensure it meets these criteria:
            - Covers the topic comprehensively
            - Flows logically from point to point
            - Addresses the needs of the target audience
            - Includes specific, actionable information
            If any criteria are not met, refine the outline accordingly.""",
            
            # Step 3: Expand the outline into a complete post
            """Based on the validated outline, write the complete blog post.
            Include all of the following:
            1. An engaging title
            2. A hook-filled introduction
            3. Well-developed main sections with subheadings
            4. A strong conclusion with a call-to-action
            5. Write in a conversational but professional tone    
            The complete post should be approximately {word_count} words."""
        ]
        
        
        # Prepare input for the chaining process
        input_data = f"""
        Topic: {topic}
        Target Audience: {target_audience}
        Approximate Word Count: {word_count}
        """
        # Initialize the parent class
        super().__init__(system_prompt, steps, input_data, model, api_key)
    
    def generate_blog_post(self, verbose=True):
        """Generate a complete blog post using the chaining process"""
        print(f"🚀 Starting blog post generation process...")
        result = self.chain(verbose=verbose)
        print(f"✅ Blog post generation complete!")
        return result
    
    def save_to_file(self, filename):
        """Save the generated blog post to a markdown file"""
        blog_post = self.generate_blog_post(verbose=False)
        with open(filename, 'w') as f:
            f.write(blog_post)
        print(f"📝 Blog post saved to {filename}")
        

In [58]:
class RouterAgent(LLMAgent):
    def __init__(self, system_prompt, choices, model, api_key, input_data):
        super().__init__(system_prompt, model, api_key)
        self.input_data = input_data
        self.choices = choices
    
    def route(self):
        routing_prompt_text=f"Analyze the following data {self.input_data.strip()} and based on the data select the most appropriate of the following options: {self.choices}"
        self.routing_prompt = LLMPrompt("user", "text", routing_prompt_text)
        chain = [self.system_prompt, self.routing_prompt.message]
        response = self.client.call(
            model=self.model,
            messages=chain,)
        return response

In [79]:
class ParallelAgent(LLMAgent):
    def __init__(self, system_prompt, tasks, instruction, model, api_key):
        self.tasks = tasks
        self.instruction = instruction
        super().__init__(system_prompt, model, api_key)

    def run_parallel(self, n_workers=3):    
        with ThreadPoolExecutor(max_workers=n_workers) as executor:
            f = lambda task : self.client.call(
                model=self.model,
                max_completion_tokens=1000,
                messages = [self.system_prompt, LLMPrompt("user", "text", f"{self.instruction}\nInput: {task}").message])
            futures = [executor.submit(f, f"{self.instruction}\nInput: {x}") for x in self.tasks]
        return [f.result() for f in futures]

## Agentic workflows

### Prompt-Chaining: Decompose a task into sequential subtasks, where each step builds on previous results

#### Example 1: Chain workflow for structured data extraction and formatting
Each step progressively transforms raw text into a formatted table

In [60]:
data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth
    23.00: customer acquisition cost
    """,
    
    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth
    23.00: customer acquisition cost""",
    
    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction
    43.00: customer acquisition cost
    34%: operating margin
    """,
    
    """Format the sorted data as a markdown table with columns, without tags or enclosing quotes:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |
    | Customer Acquisition Cost 23.00 |"""
]

In [61]:
system_prompt = "You are a helpful assistant specializing in marketing analysis."

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

In [64]:
chainerAgent = ChainerAgent(system_prompt, data_processing_steps, report, "gpt-4o", api_key)
output = chainerAgent.chain()

In [65]:
from IPython.display import display, Markdown
display(Markdown(output))

| Metric | Value |
|:--|--:|
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| New User Acquisition Cost | 43.00 |
| Operating Margin | 34% |
| Market Share | 23% |
| Customer Churn | 5% |

**Expected output**

| Metric | Value |
|:--|--:|
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| User Acquisition Cost | 43.00 |
| Operating Margin | 34% |
| Market Share | 23% |
| Previous Customer Churn | 8% |
| Customer Churn | 5% |


#### Example 2: Chain workflow to anonymise, translate and restyle text

In [66]:
anonymize_and_translate_steps = [
    """"Perform named entity recognition NER on the text, enclosing each entity in the appropriate tag.
    <example>
    Before: 'Mr. Smith went to London, where he met his wife Jane Smith and his partner Leonard Farley'
    After: 'Mr. <LASTNAME>Smith</LASTNAME> went to <CITY>London</CITY>, where he met his wife, <FIRSTNAME>Jane</FIRSTNAME><LASTNAME>Smith</LASTNAME>and his partner<FIRSTNAME>Leonard</FIRSTNAME><LASTNAME>Farley</LASTNAME>
    </example>
    """,
    
    """Anonymize the text, replacing each entity marked with a tag with a pseudonym.
    Make sure to preserve the meaning and content of sentence by replacing each entity with the same pseudonym.
    <example>
    Before: 'Mr. <LASTNAME>Smith</LASTNAME> went to <CITY>London</CITY>, where he met his wife, <FIRSTNAME>Jane</FIRSTNAME><LASTNAME>Smith</LASTNAME>and his partner<FIRSTNAME>Leonard</FIRSTNAME><LASTNAME>Farley</LASTNAME>,
    After: 'Mr. Frum went to Balberry,where he met with his wife, Ellen Frum, and his partner, Algernon Leiter.
    </example>
    """,
    """Translate the anonymized text in Italian.""",
    """Turn the Italian into verses""",
    """Turn the verse into romanesco, in the style of Belli""",
]

In [67]:
secretary_prompt = "You are a helpful secretary who values confidentiality and is adept at translations"
text = "It was a rainy day in Glasgow, when brothers Chad and John Smith embarked on their journey to Rome, where they hoped they would meet General Tso and his chicken"
chainer2 = ChainerAgent(system_prompt=secretary_prompt, steps=anonymize_and_translate_steps, inp=text, model="gpt-4o-mini", api_key=api_key)
tradit = chainer2.chain()

In [68]:
tradit

"Era ’na giornata bagnata a Balberry,  \nquanno i frate' Alex e Ben,  \nJohnson de cognome, senza pensiero,  \npartìrono in viaggio, col core sincero.\n\nVerso La Valletta, ‘na meta da sogno,  \ndove speravano, co’ tanto impegno,  \nde incontrà er Comandante Chow,  \ne er suo pollo, ‘na festa d’amore, oh, wow!"

#### Example 3: Chain workflow to create and perfect blog posts

In [69]:
class Blogger(ChainerAgent):
    def __init__(self, topic, target_audience, word_count, model="gpt-4", api_key=None):
        # Define the system prompt for blog post generation
        system_prompt = """You are a professional blog content creator who specializes in creating
        high-quality, engaging blog posts on various topics. You follow SEO best practices
        and create content that is both informative and engaging."""
        
        # Define the steps for the blog post creation pipeline
        steps = [
            # Step 1: Generate an outline
            """Create a detailed outline for a blog post on the given topic.
            Include a title, introduction, 4-6 main sections with subpoints, and conclusion.
            Format the outline with clear headings and bullet points.""",
            
            # Step 2: Validate and refine the outline
            """Review the outline and ensure it meets these criteria:
            - Covers the topic comprehensively
            - Flows logically from point to point
            - Addresses the needs of the target audience
            - Includes specific, actionable information
            If any criteria are not met, refine the outline accordingly.""",
            
            # Step 3: Expand the outline into a complete post
            """Based on the validated outline, write the complete blog post.
            Include all of the following:
            1. An engaging title
            2. A hook-filled introduction
            3. Well-developed main sections with subheadings
            4. A strong conclusion with a call-to-action
            5. Write in a conversational but professional tone    
            The complete post should be approximately {word_count} words."""
        ]
        
        
        # Prepare input for the chaining process
        input_data = f"""
        Topic: {topic}
        Target Audience: {target_audience}
        Approximate Word Count: {word_count}
        """
        
        # Initialize the parent class
        super().__init__(system_prompt, steps, input_data, model, api_key)
    
    def generate_blog_post(self, verbose=True):
        """Generate a complete blog post using the chaining process"""
        print(f"🚀 Starting blog post generation process...")
        result = self.chain(verbose=verbose)
        print(f"✅ Blog post generation complete!")
        return result
    
    def save_to_file(self, filename):
        """Save the generated blog post to a markdown file"""
        blog_post = self.generate_blog_post(verbose=False)
        with open(filename, 'w') as f:
            f.write(blog_post)
        print(f"📝 Blog post saved to {filename}")

In [71]:
blogger = Blogger(
    topic="Philosophy of Logic",
    target_audience="advanced undergraduate and graduate students in Philosophy, Computer Science and Mathematics",
    word_count=500,
    model="gpt-4o-mini",
    api_key=api_key
)
post = blogger.generate_blog_post()

🚀 Starting blog post generation process...
STEP: 0
# Outline for Blog Post: "Unraveling the Philosophy of Logic: Foundations and Implications"

## Introduction
- Brief introduction to the concept of logic and its significance in various fields.
- Importance of understanding the philosophy behind logic for advanced students in Philosophy, Computer Science, and Mathematics.
- Statement of purpose for the blog post: to explore foundational concepts, key areas of research, and the implications of the philosophy of logic.

## I. Defining Logic
- **A. Historical Perspective**  
  - Overview of the origins of logic in ancient philosophy (e.g., Aristotle's syllogisms).
  - Development through the Middle Ages to modern logic.
  
- **B. Branches of Logic**  
  - Propositional Logic vs. Predicate Logic.
  - Modal Logic and its extensions.

## II. Key Philosophical Questions in Logic
- **A. Nature of Logical Truth**  
  - What constitutes a valid argument?
  - Distinction between subjective vs. ob

In [72]:
# display(Markdown(post))

### Routing: Dynamically selects specialized LLM paths based on input characteristics

In [73]:
router = RouterAgent(
    "you are a helpful assistant specializing in routing support tickets to the appropriate department",
    model="gpt-4o-mini",
    choices=["billing", "technical", "account", "product"],
    api_key=api_key,
    input_data="""Ticket 2:
    Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah"""
)

In [74]:
route = router.route()

In [75]:
route

"Based on the content of Ticket 2, the most appropriate department to route the ticket to is **billing**. This is because the issue involves an unexpected charge on the customer's credit card and a discrepancy between the plan they thought they were on and what they were charged."

In [76]:
ticket="""Ticket 3:
----------------------------------------
Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
router = RouterAgent(
    "you are a helpful assistant specializing in routing support tickets to the appropriate department",
    model="gpt-4o-mini",
    choices=["billing", "technical", "account", "product"],
    api_key=api_key,
    input_data=ticket
)
route = router.route()
print(route)

Based on the content of Ticket 3, the request is related to exporting project data, which falls under the technical aspects of using the system. Therefore, the most appropriate option for routing this ticket is:

**technical**


### Parallelization: Distributes independent subtasks across multiple LLMs for concurrent processing

#### Example: Parallelization workflow for stakeholder impact analysis
Process impact analysis for multiple stakeholder groups concurrently

In [77]:
stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",
    
    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",
    
    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",
    
    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]
instruction =  """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities."""

In [80]:
parallel = ParallelAgent(
    "You are an expert analyst, assessing stakeholder involvement",
    stakeholders,
    instruction,
    "gpt-4o-mini",
    api_key
)

analysis = parallel.run_parallel(3)

In [81]:
analysis

['# Analysis of Market Changes Impact on Customers\n\n## Overview\nCustomers are influenced by several key market changes, including pricing strategies, technological advancements, and increased awareness of environmental issues. This analysis addresses how these factors will affect customers and outlines recommended actions to mitigate potential negative impacts and enhance customer satisfaction.\n\n## Key Impacts\n\n### 1. Price Sensitivity\n#### Potential Impact:\n- **Reduced Purchasing Power:** As market prices fluctuate, price-sensitive customers may reduce spending on non-essential items or seek cheaper alternatives, leading to potential revenue declines for businesses that do not adjust their pricing strategies.\n- **Increased Price Comparisons:** Customers may become more vigilant, comparing prices across different platforms which could increase competition among businesses.\n\n### 2. Demand for Better Technology\n#### Potential Impact:\n- **Higher Expectations:** Customers wil

## Parallel Document Analysis
Pattern: Parallelization (Sectioning)
Project: Build a document analysis tool that divides a legal contract into sections and analyzes each part simultaneously for:

- Legal compliance
- Risk assessment
- Plain language explanations
- Suggested improvements

In [ ]:
documents